In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [ ]:
import transformers
import torch
import torch.nn as nn
import pandas as pd
import torch.nn as nn
import numpy as np
from sklearn import model_selection
from sklearn import metrics
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dfx= pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
dfx.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df_positive= dfx[dfx['sentiment']=='positive'][:12500]
df_negative= dfx[dfx['sentiment']=='negative'][:12500]
dfx= pd.concat([df_positive,df_negative])
len(dfx)

25000

In [ ]:
dfx.sentiment.value_counts()

positive    12500
negative    12500
Name: sentiment, dtype: int64

In [ ]:
class config:
    MAX_LEN = 512
    #maximum number of tokens in the sentence
    TRAIN_BATCH_SIZE = 8
    VALID_BATCH_SIZE = 4
    EPOCHS = 5
    DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    BERT_PATH = "bert-base-uncased"
    MODEL_PATH = "model.pth"
    # define the tokenizer
    # we use tokenizer and model
    # from huggingface's transformers
    TOKENIZER = transformers.AutoTokenizer.from_pretrained(BERT_PATH,
                                                do_lower_case=True)

In [ ]:
t=config.TOKENIZER
t.is_fast

True

In [ ]:
dfx.sentiment = dfx.sentiment.apply(lambda x: 1 if x == "positive" else 0)
# we split the data into single training
# and validation fold
df_train, df_valid = model_selection.train_test_split(dfx,
                                                      test_size=0.3,
                                                      random_state=42,
                                                      stratify=dfx.sentiment.values)
# reset index
df_train = df_train.reset_index(drop=True)
df_valid = df_valid.reset_index(drop=True)

In [ ]:
df_train.head()

,review,sentiment
0,Unusual film about a man who befriends his soc...,1
1,"Having been a Godzilla fan for many years, Gam...",0
2,I opted to watch this film for one reason and ...,1
3,"Hello - I normally love movies. I'm 19, I have...",0
4,Don't listen to most of these people. ill give...,1


In [ ]:
import torch
class BERTDataset:
    def __init__(self, review, target):
        """
        :param review: list or numpy array of strings
        :param targets: list or numpy array which is binary
        """
        self.review = review
        self.target = target
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
    def __len__(self):
        # this returns the length of dataset
        return len(self.review)
    def __getitem__(self, item):
        # for a given item index, return a dictionary
        # of inputs
        review = str(self.review[item])
        review = " ".join(review.split())
        # here, review is a string
        inputs = self.tokenizer.encode_plus(review,
                                            add_special_tokens=True,
                                            max_length=self.max_len,
                                            pad_to_max_length=True,
                                            truncation=True)
        # ids are ids of tokens generated
        # after tokenizing reviews
        ids = inputs["input_ids"]
        # mask is 1 where we have input
        # and 0 where we have padding
        mask = inputs["attention_mask"]
        # token type ids behave the same way as
        # mask in this specific case
        # in case of two sentences, this is 0
        # for first sentence and 1 for second sentence
        token_type_ids = inputs["token_type_ids"]
        # now we return everything
        # note that ids, mask and token_type_ids
        # are all long datatypes and targets is float
        return {
                "ids": torch.tensor(
                ids, dtype=torch.long),
                "mask": torch.tensor(
                mask, dtype=torch.long),
                "token_type_ids": torch.tensor(
                token_type_ids, dtype=torch.long),
                "targets": torch.tensor(
                self.target[item], dtype=torch.float)
                }

In [ ]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        # we fetch the model from the BERT_PATH 
        self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH,return_dict=False)
        # add a dropout for regularization
        self.bert_drop = nn.Dropout(0.3)
        # a simple linear layer for output
        self.out = nn.Linear(768, 1)
    def forward(self, ids, mask, token_type_ids):
        # BERT in its default settings returns two outputs
        # last hidden state and output of bert pooler layer
        # we use the output of the pooler which is of the size
        # (batch_size, hidden_size)
        # hidden size can be 768 or 1024 depending on
        # if we are using bert base or large respectively
        # in our case, it is 768
        _, o2 = self.bert(ids,
                          attention_mask=mask,
                          token_type_ids=token_type_ids)
        # pass through dropout layer
        bo = self.bert_drop(o2)
        # pass through linear layer
        output = self.out(bo)
        # return output
        return output

In [ ]:
train_dataset = BERTDataset(review=df_train.review.values,
                                        target=df_train.sentiment.values)
# create training dataloader
train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=config.TRAIN_BATCH_SIZE,
                                                num_workers=2)

valid_dataset = BERTDataset(review=df_valid.review.values,
                                        target=df_valid.sentiment.values)
# create validation data loader
valid_data_loader = torch.utils.data.DataLoader(valid_dataset,
                                                batch_size=config.VALID_BATCH_SIZE,
                                                num_workers=1)

In [ ]:
def loss_fn(outputs, targets):
    """
    This function returns the loss.
    :param outputs: output from the model (real numbers)
    :param targets: input targets (binary)
    """
    return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))
def train_fn(data_loader, model, optimizer, device, scheduler):
    """
    This is the training function which trains for one epoch
    :param data_loader: it is the torch dataloader object
    :param model: torch model, bert in our case
    :param optimizer: adam, sgd, etc
    :param device: can be cpu or cuda
    :param scheduler: learning rate scheduler
    """
    # put the model in training mode
    model.train()
    # loop over all batches
    for d in tqdm(data_loader):
        # extract ids, token type ids and mask
        # from current batch
        # also extract targets
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]
        # move everything to specified device
        ids = ids.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        # zero-grad the optimizer
        optimizer.zero_grad()
        # pass through the model
        outputs = model(ids=ids,
                        mask=mask,
                        token_type_ids=token_type_ids)
        # calculate loss
        loss = loss_fn(outputs, targets)
        # backward step the loss
        loss.backward()
        # step optimizer
        optimizer.step()
        # step scheduler
        scheduler.step()
def eval_fn(data_loader, model, device):
    """
    this is the validation function that generates
    predictions on validation data
    :param data_loader: it is the torch dataloader object
    :param model: torch model, bert in our case
    :param device: can be cpu or cuda
    :return: output and targets
    """
    # put model in eval mode
    model.eval()
    # initialize empty lists for
    # targets and outputs
    fin_targets = []
    fin_outputs = []
    # use the no_grad scope
    # its very important else you might
    # run out of gpu memory
    with torch.no_grad():
        # this part is same as training function
        # except for the fact that there is no
        # zero_grad of optimizer and there is no loss
        # calculation or scheduler steps.
        for d in tqdm(data_loader):
            ids = d["ids"]
            token_type_ids = d["token_type_ids"]
            mask = d["mask"]
            targets = d["targets"]
            ids = ids.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)
            outputs = model(ids=ids,
                            mask=mask,
                            token_type_ids=token_type_ids)
            # convert targets to cpu and extend the final list
            targets = targets.cpu().detach()
            fin_targets.extend(targets.numpy().tolist())
            # convert outputs to cpu and extend the final list
            outputs = torch.sigmoid(outputs).cpu().detach()
            fin_outputs.extend(outputs.numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
def train(model,device):
    # load model and send it to the device
    model.to(device)
    # create parameters we want to optimize
    # we generally dont use any decay for bias
    # and weight layers
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [{
                                "params": [
                                p for n, p in param_optimizer if
                                not any(nd in n for nd in no_decay)
                                ],
                                "weight_decay": 0.001,
                                },
                                {
                                "params": [
                                p for n, p in param_optimizer if
                                any(nd in n for nd in no_decay)
                                ],
                                "weight_decay": 0.0,
                            }]
    # calculate the number of training steps
    # this is used by scheduler
    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    # AdamW optimizer
    # AdamW is the most widely used optimizer
    # for transformer based networks
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    # fetch a scheduler
    # you can also try using reduce lr on plateau
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=num_train_steps)
    # start training the epochs
    best_accuracy = 0
    for epoch in range(config.EPOCHS):
        train_fn(train_data_loader, model, optimizer, device, scheduler)
        outputs, targets = eval_fn(valid_data_loader, model, device)
        outputs = np.array(outputs) >= 0.5
        accuracy = metrics.accuracy_score(targets, outputs)
        print(f"Epoch {epoch}: Accuracy Score = {accuracy}")
        if accuracy > best_accuracy:
            torch.save(model.state_dict(), config.MODEL_PATH)
            best_accuracy = accuracy

In [ ]:
model = BERTBaseUncased()
device= config.DEVICE

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
if __name__ == "__main__":
    train(model,device)

100%|██████████| 1875/1875 [02:20<00:00, 13.36it/s]


Epoch 0: Accuracy Score = 0.9165333333333333


100%|██████████| 1875/1875 [02:20<00:00, 13.35it/s]


Epoch 1: Accuracy Score = 0.9184


100%|██████████| 1875/1875 [02:20<00:00, 13.36it/s]


Epoch 2: Accuracy Score = 0.9229333333333334


100%|██████████| 1875/1875 [02:20<00:00, 13.31it/s]


Epoch 3: Accuracy Score = 0.9353333333333333


100%|██████████| 1875/1875 [02:20<00:00, 13.36it/s]


Epoch 4: Accuracy Score = 0.9338666666666666
